In [1]:
from openai import OpenAI
import pandas as pd
import json
from pandasai import SmartDataframe
from pandasai.llm import OpenAI
llm = OpenAI(model="gpt-4o-mini", temperature=0)
# from rich.traceback import install
# install(show_locals=True)

client = OpenAI()

#### Load Data

In [2]:
# Load CSVs
df_master = pd.read_csv(r'C:\Users\AakashAI\Desktop\Repositories\Sales Agent\merged_leads.csv')
df_cleaned_leads = pd.read_csv(r'C:\Users\AakashAI\Desktop\Repositories\Sales Agent\cleaned_leads.csv')
df_leads = df_master
client = OpenAI()

FILTER_COLUMNS = [
    "Lead Number", "Lead Source", "Lead Origin", "Do Not Email",
    "Do Not Call", "Converted", "Country", "Lead Stage", "City"
]

#### Helper Functions

In [3]:
def generate_schema(dataframe: pd.DataFrame, columns: list) -> str:
    """
    Returns a JSON string describing the requested columns of a DataFrame:
    column name, data type, and up to five sample values.
    """
    schema_list = []

    for col in columns:
        if col in dataframe.columns:
            schema_list.append({
                "column_name": col,
                "dtype": str(dataframe[col].dtype),
                "examples": dataframe[col].dropna().unique()[:5].tolist()
            })

    return json.dumps(schema_list, indent=2)

def execute_pandas_query(dataframe: pd.DataFrame, query_str: str):
    """
    Evaluates a Python expression on a copy of the given DataFrame.
    Returns either a DataFrame, Series, or any resulting object.
    """
    local_vars = {"df": dataframe.copy()}

    try:
        return eval(query_str, {"__builtins__": {}}, local_vars)
    except Exception as e:
        print(f"Error: {e}")
        return None

def handle_tool_call(tool_call ):
    print("Tool call received at Handler:")
    """
    Handles the tool call by extracting the function name and arguments,
    running the appropriate function, and returning the result.
    """
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)

    if function_name == "execute_pandas_query":
        print("Executing function:", function_name)
        query_str = arguments["query_str"]
        result = execute_pandas_query(df, query_str)

        if isinstance(result, pd.DataFrame):
            return result.to_dict(orient="records")
        elif isinstance(result, pd.Series):
            return result.tolist()
        else:
            return result
    elif function_name == "nl_to_query":
        print("Converting natural language to query:", arguments["nl_query"])
        nl_query = arguments["nl_query"]
        result = nl_to_query(nl_query)
        return result
    elif function_name == "draft_sales_email_by_lead_number":
        print("Drafting sales email by lead number:", arguments["lead_number"])
        lead_number = arguments["lead_number"]
        result = draft_sales_email_by_lead_number(lead_number)
        return result    
    elif function_name == "send_email":
        print("Sending email:", arguments["email"])
        email_subject = arguments["subject"]
        email_body = arguments["body"]
        email_id = arguments["email_id"]
        result = send_email(email_subject, email_body, email_id)
        return result

#create a function
def parse_pandasai_output(output):
    import pandas as pd
    import matplotlib.pyplot as plt

    if isinstance(output, pd.DataFrame):
        print("→ Output is a DataFrame")
        display(output)  # For Jupyter or IPython
        return {"type": "dataframe", "data": output}

    elif isinstance(output, (int, float, str, bool)):
        print(f"→ Output is a {type(output).__name__}: {output}")
        return {"type": "scalar", "data": output}

    elif isinstance(output, dict):
        print("→ Output is a dictionary")
        return {"type": "dict", "data": output}

    elif isinstance(output, (list, tuple)):
        print("→ Output is a list/tuple")
        return {"type": "list", "data": output}

    elif isinstance(output, plt.Figure):
        print("→ Output is a Matplotlib Figure (chart)")
        plt.show()
        return {"type": "figure", "data": output}

    else:
        print(f"→ Output is of unexpected type: {type(output)}")
        return {"type": "unknown", "data": output}
    
def handle_tool_call(tool_call):
    print("Tool call received at Handler:")

    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)
    dataframe= df_cleaned_leads

    if function_name == "execute_pandas_query":

        print("Executing function:", function_name)

        query_str = arguments["query_str"]
        result = execute_pandas_query(dataframe, query_str)

        if isinstance(result, pd.DataFrame):
            return result.to_dict(orient="records")
        elif isinstance(result, pd.Series):
            return result.tolist()
        else:
            return result
        
    elif function_name == "nl_to_query":

        nl_query = arguments["nl_query"]
        result = nl_to_query(nl_query)

        return result
    elif function_name == "draft_sales_email_by_lead_number":

        print("Drafting sales email by lead number:", arguments["lead_number"])

        lead_number = arguments["lead_number"]
        product = arguments["product"]
        first_name = arguments["first_name"]
        last_name = arguments["last_name"]
        result = draft_sales_email_by_lead_number(lead_number, product, first_name, last_name)

        return result


#### Tool for Query DB

In [4]:
tools_secondary = [
    {
        "type": "function",
        "function": {
            "name": "execute_pandas_query",
            "description": "Run a pandas query string on a DataFrame. Use 'df' as the variable name.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query_str": {
                        "type": "string",
                        "description": "Python code string to evaluate on the DataFrame. Use 'df' as the variable name."
                    }
                },
                "required": ["query_str"]
            }
        }
    },
   
]

#### QUERY FUNCTION


In [6]:
from openai import OpenAI
client = OpenAI()
def nl_to_query_fallback(nl_query):
    df_master = pd.read_csv(r'C:\Users\AakashAI\Desktop\Repositories\Sales Agent\merged_leads.csv')
    sdf = SmartDataframe(df_master, config={
        "llm": llm,
        "custom_prompts": {
            "pandas_code": f" You are analyzing a sales leads dataset. Columns include {columns} Be careful with column names and types.",
        },
        "verbose": False
    })
    try:
        response = sdf.chat(nl_query)
        return parse_pandasai_output(response)
    except Exception as e:
        return f"Error: {e}"
def nl_to_query(user_request: str) -> pd.DataFrame:
    """
    Sends the natural language request to the LLM, which determines how to filter
    the DataFrame by calling the 'execute_pandas_query' tool. Returns the final
    filter results in Python object form. If the generated query is invalid, the 
    function retries up to 3 times.
    """
    dataframe = df_cleaned_leads
    print("Starting NL to query conversion...")
    max_retries = 3
    attempt = 0
    schema_description = generate_schema(dataframe, FILTER_COLUMNS)

    while attempt < max_retries:
        print("Entered NL query:", user_request)
        prompt = f"""
            You are an expert data analyst.

            You have access to a tool called `execute_pandas_query` that accepts a Python expression as input and runs it on a DataFrame called `df`.

            Here is the DataFrame schema:
            {schema_description}

            Below are a few one-shot examples of valid filtering queries:

            Example 1:
            User Request: "List all leads from City 'Metropolis' that have not been converted"
            Expected tool call: execute_pandas_query("df[(df['City'] == 'Metropolis') & (df['Converted'] == 0)]")

            Example 2:
            User Request: "Show me leads from Source 'Web' with a lead score above 100"
            Expected tool call: execute_pandas_query("df[(df['Lead Source'] == 'Web') & (df['Lead Score'] > 100)]")

            Example 3:
            User Request: "Find leads that are qualified and in the age group '30-35'"
            Expected tool call: execute_pandas_query("df[(df['Lead Stage'] == 'Qualified') & (df['Age'] == '30-35')]")


            Now, a user has asked to filter the DataFrame with this request:
            '{user_request}'

            You should call the tool with the appropriate pandas filtering expression using `df`.
            Understand the schema and the valid values in the columns.
            rephrase the user query with available values within the particular column.
            Strictly align with the query and do not give any other information or irrelevant details.
            Only call the tool. Do not answer the query yourself.
        """

        messages = [
            {"role": "system", "content": "You are a pandas expert."},
            {"role": "user", "content": prompt}
        ]

        response = client.chat.completions.create(
            messages=messages,
            model="gpt-4o",
            stream=False,
            tool_choice="auto",
            tools=tools_secondary,
            temperature=0
        )

        try:
            response_content = response.choices[0].message.content
            if response_content:
                print(response_content.strip())
            else:
                print("No direct LLM response. Possibly a tool call only.")
        except Exception as e:
            print(f"Error extracting LLM response: {e}")

        # Extract the first tool call
        try:
            tool_call = response.choices[0].message.tool_calls[0]
            print(f"Tool call from Query Function: {tool_call}")
        except Exception as e:
            print("Failed to extract a tool call from the LLM response:", e)
            attempt += 1
            continue  # Retry the LLM call

        # Validate query by attempting to execute it
        try:
            tool_result = handle_tool_call(tool_call)
            print("tool result from Query Function:", tool_result)
            return tool_result  # Return if successful
        except Exception as e:
            print(f"Query execution failed on attempt {attempt+1}: {e}")
            attempt += 1
            print("Retrying query generation...\n")
            # If the LLM response is empty or invalid, retry with pandas query generation
            return nlp_to_query_fallback(user_request)


    # If all attempts fail, raise an exception
    raise Exception("Failed to generate a valid query after maximum retry attempts.")

# Example usage:
nl_query = "How many leads are from landing page?"
result = nl_to_query(nl_query)


Starting NL to query conversion...
Entered NL query: How many leads are from landing page?
No direct LLM response. Possibly a tool call only.
Tool call from Query Function: ChatCompletionMessageToolCall(id='call_r5N3JSa08f86LgM6UxNfIvRO', function=Function(arguments='{"query_str":"df[df[\'Lead Origin\'] == \'Landing Page Submission\'].shape[0]"}', name='execute_pandas_query'), type='function')
Tool call received at Handler:
Executing function: execute_pandas_query
tool result from Query Function: 19


#### DRAFT EMAIL

In [7]:
def draft_sales_email_by_lead_number(lead_number: int, product: str, first_name: str, last_name: str) -> str:
    """
    Drafts a personalized sales email for a given lead number by looking up the lead data from the DataFrame.
    
    Parameters:
      lead_number (int): The unique identifier of the lead.
      product (str): The product or service to pitch.
    
    Returns:
      str: The drafted sales email.
    
    Raises:
      Exception: If the lead number is not found.
    """
    print("entered email drafting function")
    agent_name = "Jane Doe"  # Placeholder for the agent's name
    agent_company = "Tech Solutions"  # Placeholder for the agent's company
    agent_contact_information = "jane.doe@xyz.com"

    email_body ={
        ""
    }

    # Look up the lead in the DataFrame
    lead_data = df_leads[df_leads['Lead Number_x'] == lead_number]
    
    if lead_data.empty:
        raise Exception(f"Lead with number {lead_number} not found.")
    
    # Assuming each Lead Number is unique, get the first row
    lead = lead_data.iloc[0].to_dict()
    
    # Create the email subject and body using the lead details
    email_subject = f"Discover How {product} Can Benefit {first_name, last_name}"
    email_body = (
        f"Hi {first_name} {last_name},\n\n"
        f"I hope you're doing well. My name is {agent_name} and I work as a Sales Engineer at {agent_company}.\n\n"
        f"I'm reaching out because I believe our {product} can significantly help {lead.get('Company', 'your company')} meet its goals. "
        "We have a strong track record of helping organizations in your industry optimize their processes and drive success.\n\n"
        "I'd love to arrange a brief call to discuss how we can support your goals and explore further benefits of our solution.\n\n"
        "Looking forward to hearing from you.\n\n"
        "Best regards,\n"
        f"{agent_name}\n"
        f"{agent_company}\n"
        f"{agent_contact_information}"
    )
    
    return f"Subject: {email_subject}\n\n{email_body}"
print(draft_sales_email_by_lead_number(lead_number=651812, product="AI Solutions", first_name="John", last_name="Doe"))

entered email drafting function
Subject: Discover How AI Solutions Can Benefit ('John', 'Doe')

Hi John Doe,

I hope you're doing well. My name is Jane Doe and I work as a Sales Engineer at Tech Solutions.

I'm reaching out because I believe our AI Solutions can significantly help nehamenon26@rediffmail.com meet its goals. We have a strong track record of helping organizations in your industry optimize their processes and drive success.

I'd love to arrange a brief call to discuss how we can support your goals and explore further benefits of our solution.

Looking forward to hearing from you.

Best regards,
Jane Doe
Tech Solutions
jane.doe@xyz.com


#### SEND EMAIL (DUMMY)

In [8]:
def send_email(subject: str, body: str, email_id: str) -> str:
    """
    Simulates sending an email with the given subject and body to the specified email id.
    
    Parameters:
      subject (str): The subject of the email.
      body (str): The body of the email.
      email_id (str): The recipient's email address.
      
    Returns:
      str: A confirmation message indicating that the email was "sent".
    """
    # Simulation: In a real system this would interface with an email API.
    confirmation_message = f"Dummy email sent to {email_id} with subject '{subject} and Body {body}'."
    return confirmation_message

#### ORCHESTRATOR SDR AGENT

In [9]:
SDR_ASSISTANT_INSTRUCTIONS = """
You are a friendly and efficient assistant for a Sales Development Representative (SDR).
ALWAYS CHECK FOR INFORMATION IN THE CONVERSATION HISTORY BEFORE ASKING THE SDR FOR MORE INFORMATION. TOOLS MIGHT HAVE BEEN CALLED ALREADY.

Your job is to support the SDR in identifying high-quality leads and drafting effective, personalized sales emails.
You help the SDR save time by interpreting their natural language requests and converting them into meaningful actions using the tools available to you.

You have access to three tools:

1. nl_to_query
   - Use this tool to retrieve relevant leads from the database.
   - NEVER write pandas code yourself. Instead, rephrase the SDR’s request in natural language and call this tool.
   - Use this tool when the SDR asks to find or filter leads (e.g., "Show me leads from Mumbai who haven’t converted yet").
   - ALWAYS ASK FOR MISSING INFORMATION TO COMPLETE THE REQUEST.

2. draft_sales_email_by_lead_number
   - Use this tool to generate a personalized sales email for a specific lead.
   - You’ll need the lead number, product name, and the lead's first and last name.
   - The email will be drafted using data from the leads database to make it more relevant and engaging.
   - DO NOT call the nl_to_query tool for email drafting requests.
   - ALWAYS ASK FOR MISSING INFORMATION TO COMPLETE THE REQUEST if any details (e.g., lead number, first name, last name, or product) are missing.

3. send_email
   - Use this tool to send an email.
   - It takes in the subject, body, and recipient's email address, and returns a confirmation message.
   - After the SDR confirms that the drafted email is good, you must call this tool to "send" the email.

What you can help the SDR with:
- Finding specific leads (e.g., "Show me leads from Mumbai who haven’t converted yet")
- Filtering leads by stage, score, city, source, or conversion status
- Identifying hot leads based on attributes (e.g., score > 100, lead grade = A/B)
- Drafting follow-up or outreach emails customized to the lead
- Suggesting lead prioritization based on score, source, or engagement

Important Instructions:
- Always use nl_to_query to query the leads database. NEVER attempt to write the query directly.
- ALWAYS ASK FOR MISSING INFORMATION TO COMPLETE THE REQUEST.
- If the SDR wants to send a sales email, gather the lead number, the lead's first and last name, and the product before proceeding.
- DO NOT call nl_to_query for email drafting requests.
- If the SDR's request is vague, ask clarifying questions to ensure you understand their needs before proceeding.
- After the SDR confirms that the email draft is acceptable, collect the recipient's email address (if not provided) and call send_dummy_email with the subject, body, and email address.
- If you are unsure or need more clarification, politely ask the SDR for specifics before proceeding.

Stay helpful, relevant, and focused on helping the SDR close more leads efficiently.
"""
 

memory = {
    "sdr" : {
        "messages": [
        {"role": "system", "content": SDR_ASSISTANT_INSTRUCTIONS},
    ]
    }
}
tools_primary = [{
  "type": "function",
  "function": {
    "name": "nl_to_query",
    "description": "Convert a natural language query into a valid pandas filtering expression.",
    "parameters": {
      "type": "object",
      "properties": {
        "nl_query": {
          "type": "string",
          "description": "The user's natural language filter request (e.g., 'show leads from Delhi with conversion 0')."
        }
      },
      "required": ["nl_query"]
    }
  }
},
 {
        "type": "function",
        "function": {
            "name": "draft_sales_email_by_lead_number",
            "description": (
                "Drafts a personalized sales email for a given lead number by looking up the lead data "
                "from the DataFrame and injecting a product pitch and lead's name."
            ),
            "parameters": {
                "type": "object",
                "properties": {
                    "lead_number": {
                        "type": "integer",
                        "description": "Unique integer identifier for the lead."
                    },
                    "product": {
                        "type": "string",
                        "description": "The product or service to pitch."
                    },
                    "first_name": {
                        "type": "string",
                        "description": "First name of the lead/contact person."
                    },
                    "last_name": {
                        "type": "string",
                        "description": "Last name of the lead/contact person."
                    }
                },
                "required": ["lead_number", "product", "first_name", "last_name"]
            }
        }
    },

    {
        "type": "function",
        "function": {
            "name": "send_email",
            "description": (
                "Sends an email with the provided subject, body, and recipient email address, "
                "then returns a confirmation message."
            ),
            "parameters": {
                "type": "object",
                "properties": {
                    "subject": {
                        "type": "string",
                        "description": "The subject of the email."
                    },
                    "body": {
                        "type": "string",
                        "description": "The content of the email body."
                    },
                    "email_id": {
                        "type": "string",
                        "description": "The recipient's email address."
                    }
                },
                "required": ["subject", "body", "email_id"]
            }
        }
    }
]

In [10]:
def sdr(user_message):
    print("Entered SDR function")
    # print("Prompt for OpenAI:", prompt)
    memory['sdr']['messages'].append(
        {"role": "user", "content": user_message}
    )
    response = client.chat.completions.create(
            messages=memory['sdr']['messages'],
            model="o3-mini",
            stream=False,
            tool_choice="auto",
            tools=tools_primary,
            # temperature=0, # Keep temperature low for deterministic SQL generation
        )
    print((response))
    if not response.choices[0].message.tool_calls:
        memory['sdr']['messages'].append(
            {
                "role": "assistant",
                "content": response.choices[0].message.content
            }
        )
        return response.choices[0].message.content

    tool_call = response.choices[0].message.tool_calls[0]
    print(f"Tool call from SDR: {tool_call}")
    tool_result = handle_tool_call(tool_call)
    print("Tool result from SDR:", tool_result)

    memory['sdr']['messages']=[
                *memory['sdr']['messages'],
                {
                    "role": "assistant",
                    "tool_calls": [tool_call]
                },
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": "nl_to_query",
                    "content": json.dumps(tool_result)
                }
            ]
    followup = client.chat.completions.create(
            model="gpt-4o",
            messages=memory['sdr']['messages']
        )
    return followup.choices[0].message.content
    

In [20]:
user_input = input()
sdr(user_input)

Entered SDR function
ChatCompletion(id='chatcmpl-BLYxFBv8f0wOBvjxubHAszdaI7c1J', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Here are the details for lead number 659357:\n\n• Lead Number: 659357  \n• Lead Source: Google  \n• Lead Origin: Landing Page Submission  \n• Do Not Email: No  \n• Do Not Call: No  \n• Converted: 0  \n• Country: India  \n• Mobile Number: Not provided  \n• Lead Stage: Unreachable  \n• Lead Grade: Not provided  \n• Lead Score: 120  \n• Age: Select  \n• Email: Yogeshsadarang@yahoo.in  \n• City: Mumbai\n\nWould you like to take any further actions for this lead, such as drafting a personalized email or scheduling a follow-up?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1744478953, model='o3-mini-2025-01-31', object='chat.completion', service_tier='default', system_fingerprint='fp_99801fd0cf', usage=CompletionUsage(completion_tokens=335, prompt_to

'Here are the details for lead number 659357:\n\n• Lead Number: 659357  \n• Lead Source: Google  \n• Lead Origin: Landing Page Submission  \n• Do Not Email: No  \n• Do Not Call: No  \n• Converted: 0  \n• Country: India  \n• Mobile Number: Not provided  \n• Lead Stage: Unreachable  \n• Lead Grade: Not provided  \n• Lead Score: 120  \n• Age: Select  \n• Email: Yogeshsadarang@yahoo.in  \n• City: Mumbai\n\nWould you like to take any further actions for this lead, such as drafting a personalized email or scheduling a follow-up?'

In [16]:
memory

{'sdr': {'messages': [{'role': 'system',
    'content': '\nYou are a friendly and efficient assistant for a Sales Development Representative (SDR).\nALWAYS CHECK FOR INFORMATION IN THE CONVERSATION HISTORY BEFORE ASKING THE SDR FOR MORE INFORMATION. TOOLS MIGHT HAVE BEEN CALLED ALREADY.\n\nYour job is to support the SDR in identifying high-quality leads and drafting effective, personalized sales emails.\nYou help the SDR save time by interpreting their natural language requests and converting them into meaningful actions using the tools available to you.\n\nYou have access to three tools:\n\n1. nl_to_query\n   - Use this tool to retrieve relevant leads from the database.\n   - NEVER write pandas code yourself. Instead, rephrase the SDR’s request in natural language and call this tool.\n   - Use this tool when the SDR asks to find or filter leads (e.g., "Show me leads from Mumbai who haven’t converted yet").\n   - ALWAYS ASK FOR MISSING INFORMATION TO COMPLETE THE REQUEST.\n\n2. draft_s